In [1]:
# 📓 01_data_cleaning.ipynb
# Step-by-Step Data Cleaning Process

import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🚀 STARTING DATA CLEANING PROCESS")
print("=" * 60)

🚀 STARTING DATA CLEANING PROCESS


In [2]:
print("🔹 STEP 1: Import and Load Data")
print("-" * 40)

# 1.1 Import essential libraries
print("✅ Importing libraries: pandas, numpy, datetime")

# 1.2 Load dataset
data_path = "../data/Financials.json"
print(f"📁 Loading data from: {data_path}")

try:
    with open(data_path, 'r') as file:
        data = json.load(file)
    
    df = pd.DataFrame(data)
    print(f"✅ Successfully loaded data")
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    raise

# 1.3 Display first few rows and shape
print(f"📊 Dataset Shape: {df.shape} (rows, columns)")
print("\nFirst 5 rows:")
display(df.head())

print(f"\n📈 Total Records: {len(df)}")
print(f"📊 Total Columns: {len(df.columns)}")

🔹 STEP 1: Import and Load Data
----------------------------------------
✅ Importing libraries: pandas, numpy, datetime
📁 Loading data from: ../data/Financials.json
✅ Successfully loaded data
📊 Dataset Shape: (700, 16) (rows, columns)

First 5 rows:


,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,"$1,618.50",$3.00,$20.00,"$32,370.00",$-,"$32,370.00","$16,185.00","$16,185.00",01/01/2014,1,January,2014
1,Government,Germany,Carretera,None,"$1,321.00",$3.00,$20.00,"$26,420.00",$-,"$26,420.00","$13,210.00","$13,210.00",01/01/2014,1,January,2014
2,Midmarket,France,Carretera,None,"$2,178.00",$3.00,$15.00,"$32,670.00",$-,"$32,670.00","$21,780.00","$10,890.00",01/06/2014,6,June,2014
3,Midmarket,Germany,Carretera,None,$888.00,$3.00,$15.00,"$13,320.00",$-,"$13,320.00","$8,880.00","$4,440.00",01/06/2014,6,June,2014
4,Midmarket,Mexico,Carretera,None,"$2,470.00",$3.00,$15.00,"$37,050.00",$-,"$37,050.00","$24,700.00","$12,350.00",01/06/2014,6,June,2014



📈 Total Records: 700
📊 Total Columns: 16


In [3]:
print("🔹 STEP 2: Data Consistency Rules")
print("-" * 40)

def check_consistency_rules(df):
    """
    Check data consistency rules for different column types
    """
    consistency_report = {}
    
    for column in df.columns:
        print(f"\n📋 Checking column: {column}")
        print(f"   Data type: {df[column].dtype}")
        
        # Get non-null values for analysis
        non_null_values = df[column].dropna()
        
        if len(non_null_values) == 0:
            print("   ⚠️  Column is empty")
            consistency_report[column] = {"status": "empty", "issues": []}
            continue
            
        # Rule 1: Check for numeric columns (float, int)
        if pd.api.types.is_numeric_dtype(df[column]):
            print("   🔢 Numeric column detected")
            
            # Check if all values are actually numeric
            numeric_issues = []
            
            # Check for infinite values
            infinite_count = np.isinf(df[column]).sum()
            if infinite_count > 0:
                numeric_issues.append(f"Contains {infinite_count} infinite values")
                
            # Check for extreme outliers (beyond 6 standard deviations)
            if len(non_null_values) > 1:
                z_scores = np.abs((non_null_values - non_null_values.mean()) / non_null_values.std())
                extreme_outliers = (z_scores > 6).sum()
                if extreme_outliers > 0:
                    numeric_issues.append(f"Contains {extreme_outliers} extreme outliers")
            
            consistency_report[column] = {
                "type": "numeric",
                "status": "valid" if len(numeric_issues) == 0 else "issues",
                "issues": numeric_issues,
                "stats": {
                    "min": non_null_values.min(),
                    "max": non_null_values.max(),
                    "mean": non_null_values.mean(),
                    "null_count": df[column].isnull().sum()
                }
            }
            
        # Rule 2: Check for categorical columns (object)
        elif df[column].dtype == 'object':
            print("   📝 Categorical column detected")
            
            categorical_issues = []
            
            # Check for mixed data types
            type_counts = non_null_values.apply(type).value_counts()
            if len(type_counts) > 1:
                categorical_issues.append(f"Mixed types: {dict(type_counts)}")
            
            # Check for numeric values in categorical columns
            numeric_values = pd.to_numeric(non_null_values, errors='coerce')
            numeric_count = numeric_values.notna().sum()
            if numeric_count > 0:
                categorical_issues.append(f"Contains {numeric_count} numeric values")
            
            # Check for inconsistent formatting
            # Look for values with special characters that might indicate mixed content
            special_char_pattern = r'[0-9]+\s*[a-zA-Z]+|[a-zA-Z]+\s*[0-9]+'
            mixed_content = non_null_values.astype(str).str.contains(special_char_pattern, na=False).sum()
            if mixed_content > 0:
                categorical_issues.append(f"Contains {mixed_content} mixed alphanumeric values")
            
            consistency_report[column] = {
                "type": "categorical",
                "status": "valid" if len(categorical_issues) == 0 else "issues",
                "issues": categorical_issues,
                "stats": {
                    "unique_values": len(non_null_values.unique()),
                    "most_frequent": non_null_values.mode().iloc[0] if len(non_null_values.mode()) > 0 else None,
                    "null_count": df[column].isnull().sum()
                }
            }
            
        # Rule 3: Check for datetime columns
        elif pd.api.types.is_datetime64_any_dtype(df[column]):
            print("   📅 Datetime column detected")
            
            datetime_issues = []
            
            # Check for invalid dates
            if pd.api.types.is_datetime64_any_dtype(df[column]):
                # Check for far future/past dates (potential errors)
                if len(non_null_values) > 0:
                    min_date = non_null_values.min()
                    max_date = non_null_values.max()
                    
                    # Check for unrealistic dates (before 1900 or after 2100)
                    if min_date.year < 1900:
                        datetime_issues.append(f"Contains dates before 1900: {min_date}")
                    if max_date.year > 2100:
                        datetime_issues.append(f"Contains dates after 2100: {max_date}")
            
            consistency_report[column] = {
                "type": "datetime",
                "status": "valid" if len(datetime_issues) == 0 else "issues",
                "issues": datetime_issues,
                "stats": {
                    "date_range": f"{min_date} to {max_date}" if len(non_null_values) > 0 else "N/A",
                    "null_count": df[column].isnull().sum()
                }
            }
            
        else:
            print(f"   🔍 Other data type: {df[column].dtype}")
            consistency_report[column] = {
                "type": "other",
                "status": "unknown",
                "issues": ["Unclassified data type"],
                "stats": {"null_count": df[column].isnull().sum()}
            }
    
    return consistency_report

def print_consistency_summary(consistency_report):
    """
    Print a summary of consistency check results
    """
    print("\n" + "="*60)
    print("📊 CONSISTENCY CHECK SUMMARY")
    print("="*60)
    
    total_columns = len(consistency_report)
    valid_columns = sum(1 for col_report in consistency_report.values() 
                       if col_report['status'] == 'valid')
    issue_columns = sum(1 for col_report in consistency_report.values() 
                       if col_report['status'] == 'issues')
    
    print(f"📈 Total Columns Checked: {total_columns}")
    print(f"✅ Valid Columns: {valid_columns}")
    print(f"❌ Columns with Issues: {issue_columns}")
    
    # Print detailed issues
    print("\n🔍 DETAILED ISSUES:")
    print("-" * 40)
    
    for column, report in consistency_report.items():
        if report['status'] != 'valid' and report['status'] != 'empty':
            print(f"\n📋 {column} ({report['type']}):")
            for issue in report['issues']:
                print(f"   ⚠️  {issue}")
            
            # Print additional stats
            if 'stats' in report:
                stats = report['stats']
                if 'null_count' in stats and stats['null_count'] > 0:
                    print(f"   📊 Null values: {stats['null_count']}")

# Run consistency check
print("🔍 Starting consistency checks...")
consistency_report = check_consistency_rules(df)

# Print summary
print_consistency_summary(consistency_report)

# Additional: Check for mixed types in object columns
print("\n" + "="*60)
print("🔍 DEEP CHECK: Mixed Types in Object Columns")
print("="*60)

def check_mixed_types_deep(df):
    """Deep check for mixed types in object columns"""
    object_columns = df.select_dtypes(include=['object']).columns
    
    for column in object_columns:
        print(f"\n🔍 Deep checking: {column}")
        
        # Sample non-null values to check types
        sample_values = df[column].dropna().head(10)
        
        if len(sample_values) == 0:
            print("   ✅ No data to check")
            continue
            
        # Check actual Python types
        type_dict = {}
        for val in sample_values:
            val_type = type(val).__name__
            type_dict[val_type] = type_dict.get(val_type, 0) + 1
        
        if len(type_dict) > 1:
            print(f"   ❌ MIXED TYPES FOUND: {type_dict}")
        else:
            print(f"   ✅ Consistent types: {list(type_dict.keys())[0]}")

check_mixed_types_deep(df)

print("\n🎯 CONSISTENCY CHECK COMPLETED!")

🔹 STEP 2: Data Consistency Rules
----------------------------------------
🔍 Starting consistency checks...

📋 Checking column: Segment
   Data type: object
   📝 Categorical column detected

📋 Checking column: Country
   Data type: object
   📝 Categorical column detected

📋 Checking column: Product
   Data type: object
   📝 Categorical column detected

📋 Checking column: Discount Band
   Data type: object
   📝 Categorical column detected

📋 Checking column: Units Sold
   Data type: object
   📝 Categorical column detected

📋 Checking column: Manufacturing Price
   Data type: object
   📝 Categorical column detected

📋 Checking column: Sale Price
   Data type: object
   📝 Categorical column detected

📋 Checking column: Gross Sales
   Data type: object
   📝 Categorical column detected

📋 Checking column: Discounts
   Data type: object
   📝 Categorical column detected

📋 Checking column: Sales
   Data type: object
   📝 Categorical column detected

📋 Checking column: COGS
   Data type: object

In [5]:
print("\n🔹 STEP 2: Inspect and Understand Dataset")
print("-" * 40)

# 2.1 Print column names and data types
print("📋 Column Names and Data Types:")
print(df.dtypes)

# 2.2 Identify column types
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
object_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"\n🔢 Numeric Columns: {numeric_cols}")
print(f"🏷️ Object Columns: {object_cols}")

# 2.3 Check for null or missing values
print("\n🔍 Missing Values Check:")
missing_data = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100

missing_info = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})
print(missing_info[missing_info['Missing Count'] > 0])

if missing_info['Missing Count'].sum() == 0:
    print("✅ No missing values found!")

# 2.4 Review summary statistics
print("\n📊 Summary Statistics for Numeric Columns:")
print(df.describe())

# Additional info about categorical data
print("\n🏷️ Categorical Columns Info:")
for col in object_cols:
    if col != 'Date':  # Exclude date column for now
        print(f"{col}: {df[col].nunique()} unique values")
        print(f"   Samples: {df[col].unique()[:5]}")  # Show first 5 unique values


🔹 STEP 2: Inspect and Understand Dataset
----------------------------------------
📋 Column Names and Data Types:
Segment                object
Country                object
Product                object
Discount Band          object
Units Sold             object
Manufacturing Price    object
Sale Price             object
Gross Sales            object
Discounts              object
Sales                  object
COGS                   object
Profit                 object
Date                   object
Month Number            int64
Month Name             object
Year                    int64
dtype: object

🔢 Numeric Columns: ['Month Number', 'Year']
🏷️ Object Columns: ['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold', 'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts', 'Sales', 'COGS', 'Profit', 'Date', 'Month Name']

🔍 Missing Values Check:
Empty DataFrame
Columns: [Missing Count, Missing Percentage]
Index: []
✅ No missing values found!

📊 Summary Statistics for 

In [6]:
print("\n🔹 STEP 3: Clean Column Names")
print("-" * 40)

# Create a copy for cleaning
df_clean = df.copy()

print("📝 Original column names:")
print(list(df_clean.columns))

# 3.1 Rename columns for consistency
column_mapping = {
    'Discount Band': 'Discount_Band',
    'Units Sold': 'Units_Sold',
    'Manufacturing Price': 'Manufacturing_Price',
    'Sale Price': 'Sale_Price',
    'Gross Sales': 'Gross_Sales',
    'Month Number': 'Month_Number',
    'Month Name': 'Month_Name'
}

df_clean.rename(columns=column_mapping, inplace=True)

print("✅ Cleaned column names:")
print(list(df_clean.columns))

# Verify changes
print(f"\n📊 Columns renamed: {len(column_mapping)}")


🔹 STEP 3: Clean Column Names
----------------------------------------
📝 Original column names:
['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold', 'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts', 'Sales', 'COGS', 'Profit', 'Date', 'Month Number', 'Month Name', 'Year']
✅ Cleaned column names:
['Segment', 'Country', 'Product', 'Discount_Band', 'Units_Sold', 'Manufacturing_Price', 'Sale_Price', 'Gross_Sales', 'Discounts', 'Sales', 'COGS', 'Profit', 'Date', 'Month_Number', 'Month_Name', 'Year']

📊 Columns renamed: 7


In [7]:
print("\n🔹 STEP 4: Clean Financial Values")
print("-" * 40)

# 4.1 Define currency columns to clean
currency_columns = [
    'Units_Sold', 'Manufacturing_Price', 'Sale_Price', 
    'Gross_Sales', 'Discounts', 'Sales', 'COGS', 'Profit'
]

print("💰 Cleaning currency columns:")

def clean_currency_value(value):
    """Clean individual currency values"""
    if isinstance(value, str):
        # Remove $, commas, spaces, and handle negative values
        cleaned = value.replace('$', '').replace(',', '').replace(' ', '')
        cleaned = cleaned.replace('$-', '-')  # Handle negative format
        return cleaned
    return value

# 4.2 Clean each currency column
for col in currency_columns:
    if col in df_clean.columns:
        print(f"   Cleaning {col}...")
        
        # Before cleaning
        sample_before = df_clean[col].iloc[0] if len(df_clean) > 0 else "N/A"
        
        # Apply cleaning
        df_clean[col] = df_clean[col].apply(clean_currency_value)
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        
        # After cleaning
        sample_after = df_clean[col].iloc[0] if len(df_clean) > 0 else "N/A"
        
        print(f"     Before: {sample_before} → After: {sample_after}")

print("✅ All currency columns cleaned and converted to numeric")

# 4.3 Verify numeric conversion
print("\n📊 Data types after cleaning:")
print(df_clean[currency_columns].dtypes)


🔹 STEP 4: Clean Financial Values
----------------------------------------
💰 Cleaning currency columns:
   Cleaning Units_Sold...
     Before:  $1,618.50  → After: 1618.5
   Cleaning Manufacturing_Price...
     Before:  $3.00  → After: 3.0
   Cleaning Sale_Price...
     Before:  $20.00  → After: 20.0
   Cleaning Gross_Sales...
     Before:  $32,370.00  → After: 32370.0
   Cleaning Discounts...
     Before:  $-    → After: nan
   Cleaning Sales...
     Before:  $32,370.00  → After: 32370.0
   Cleaning COGS...
     Before:  $16,185.00  → After: 16185.0
   Cleaning Profit...
     Before:  $16,185.00  → After: 16185.0
✅ All currency columns cleaned and converted to numeric

📊 Data types after cleaning:
Units_Sold             float64
Manufacturing_Price    float64
Sale_Price             float64
Gross_Sales            float64
Discounts              float64
Sales                  float64
COGS                   float64
Profit                 float64
dtype: object


In [9]:
print("\n🔹 STEP 5: Handle Missing or Invalid Data")
print("-" * 40)

# 5.1 Check for missing values after cleaning
print("🔍 Checking for missing values after cleaning...")
missing_after_clean = df_clean.isnull().sum()

if missing_after_clean.sum() > 0:
    print("⚠️ Missing values found after cleaning:")
    missing_cols = missing_after_clean[missing_after_clean > 0]
    for col, count in missing_cols.items():
        print(f"   {col}: {count} missing values ({count/len(df_clean)*100:.2f}%)")
    
    # 5.2 Handle missing values
    print("\n🛠️ Handling missing values...")
    
    # For numeric columns, fill with 0 or median
    for col in currency_columns:
        if col in df_clean.columns and df_clean[col].isnull().sum() > 0:
            fill_value = 0  # or df_clean[col].median() for more robust filling
            df_clean[col].fillna(fill_value, inplace=True)
            print(f"   Filled {col} with {fill_value}")
    
else:
    print("✅ No missing values found after cleaning!")

# 5.3 Check for invalid values (e.g., negative units sold)
print("\n🔎 Checking for invalid values...")
for col in ['Units_Sold', 'Gross_Sales', 'Profit']:
    if col in df_clean.columns:
        negative_count = (df_clean[col] < 0).sum()
        if negative_count > 0:
            print(f"   ⚠️ {col}: {negative_count} negative values")
        else:
            print(f"   ✅ {col}: No negative values")

# 5.4 Final missing values check
final_missing = df_clean.isnull().sum().sum()
print(f"\n🎯 Final missing values count: {final_missing}")


🔹 STEP 5: Handle Missing or Invalid Data
----------------------------------------
🔍 Checking for missing values after cleaning...
✅ No missing values found after cleaning!

🔎 Checking for invalid values...
   ✅ Units_Sold: No negative values
   ✅ Gross_Sales: No negative values
   ✅ Profit: No negative values

🎯 Final missing values count: 0


In [10]:
print("\n🔹 STEP 6: Format Date Columns")
print("-" * 40)

# 6.1 Convert Date column to datetime
print("📅 Converting Date column to datetime...")

if 'Date' in df_clean.columns:
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%m/%d/%Y', errors='coerce')
    
    # Check for invalid dates
    invalid_dates = df_clean['Date'].isnull().sum()
    if invalid_dates > 0:
        print(f"⚠️ {invalid_dates} invalid dates found and set to NaT")
    
    print(f"✅ Date range: {df_clean['Date'].min()} to {df_clean['Date'].max()}")

# 6.2 Ensure Year, Month_Number, Month_Name are consistent
print("\n🔍 Verifying date consistency...")

if 'Year' in df_clean.columns and 'Date' in df_clean.columns:
    # Extract year from Date and compare with existing Year column
    df_clean['Year_From_Date'] = df_clean['Date'].dt.year
    
    mismatched_years = (df_clean['Year'] != df_clean['Year_From_Date']).sum()
    print(f"📊 Year consistency check: {mismatched_years} mismatched records")
    
    # Use the Year from Date column as it's more reliable
    df_clean['Year'] = df_clean['Year_From_Date']
    df_clean.drop('Year_From_Date', axis=1, inplace=True)

# 6.3 Sort dataset chronologically
print("\n📈 Sorting dataset by date...")
df_clean.sort_values('Date', inplace=True)
df_clean.reset_index(drop=True, inplace=True)

print("✅ Dataset sorted chronologically")
print(f"📅 Final date range: {df_clean['Date'].min()} to {df_clean['Date'].max()}")


🔹 STEP 6: Format Date Columns
----------------------------------------
📅 Converting Date column to datetime...
✅ Date range: 2013-01-09 00:00:00 to 2014-01-12 00:00:00

🔍 Verifying date consistency...
📊 Year consistency check: 0 mismatched records

📈 Sorting dataset by date...
✅ Dataset sorted chronologically
📅 Final date range: 2013-01-09 00:00:00 to 2014-01-12 00:00:00


In [11]:
print("\n🔹 STEP 7: Feature Engineering")
print("-" * 40)

print("🎯 Creating new KPIs for analysis...")

# 7.1 Profit Margin
if 'Profit' in df_clean.columns and 'Sales' in df_clean.columns:
    df_clean['Profit_Margin'] = (df_clean['Profit'] / df_clean['Sales']) * 100
    print(f"✅ Created Profit_Margin: {df_clean['Profit_Margin'].mean():.2f}% average")

# 7.2 Cost Ratio
if 'COGS' in df_clean.columns and 'Sales' in df_clean.columns:
    df_clean['Cost_Ratio'] = (df_clean['COGS'] / df_clean['Sales']) * 100
    print(f"✅ Created Cost_Ratio: {df_clean['Cost_Ratio'].mean():.2f}% average")

# 7.3 Revenue per Unit
if 'Sales' in df_clean.columns and 'Units_Sold' in df_clean.columns:
    df_clean['Revenue_per_Unit'] = df_clean['Sales'] / df_clean['Units_Sold']
    print(f"✅ Created Revenue_per_Unit: ${df_clean['Revenue_per_Unit'].mean():.2f} average")

# 7.4 Discount Percentage (if applicable)
if 'Discounts' in df_clean.columns and 'Gross_Sales' in df_clean.columns:
    df_clean['Discount_Percentage'] = (df_clean['Discounts'] / df_clean['Gross_Sales']) * 100
    print(f"✅ Created Discount_Percentage: {df_clean['Discount_Percentage'].mean():.2f}% average")

print(f"\n📊 New columns created: {[col for col in df_clean.columns if col not in df.columns]}")


🔹 STEP 7: Feature Engineering
----------------------------------------
🎯 Creating new KPIs for analysis...
✅ Created Profit_Margin: 28.46% average
✅ Created Cost_Ratio: 72.10% average
✅ Created Revenue_per_Unit: $109.80 average
✅ Created Discount_Percentage: 7.33% average

📊 New columns created: ['Discount_Band', 'Units_Sold', 'Manufacturing_Price', 'Sale_Price', 'Gross_Sales', 'Month_Number', 'Month_Name', 'Profit_Margin', 'Cost_Ratio', 'Revenue_per_Unit', 'Discount_Percentage']


In [12]:
print("\n🔹 STEP 8: Validate Data Quality")
print("-" * 40)

print("🔍 Running final data quality checks...")

# 8.1 Recheck data types
print("\n📋 Final Data Types:")
print(df_clean.dtypes)

# 8.2 Check for duplicates
duplicates = df_clean.duplicated().sum()
print(f"\n🔍 Duplicate rows: {duplicates}")

if duplicates > 0:
    print("⚠️ Removing duplicate rows...")
    df_clean = df_clean.drop_duplicates()
    print(f"✅ Removed {duplicates} duplicates")

# 8.3 Verify logical consistency
print("\n✅ Logical Consistency Checks:")

# Profit ≈ Sales - COGS
if all(col in df_clean.columns for col in ['Profit', 'Sales', 'COGS']):
    calculated_profit = df_clean['Sales'] - df_clean['COGS']
    profit_diff = (df_clean['Profit'] - calculated_profit).abs().mean()
    print(f"   Profit consistency: Average difference = ${profit_diff:.2f}")

# Sales > COGS (in most cases)
sales_greater = (df_clean['Sales'] > df_clean['COGS']).sum()
sales_ratio = sales_greater / len(df_clean) * 100
print(f"   Sales > COGS: {sales_ratio:.1f}% of records")

# 8.4 Final missing values check
final_missing = df_clean.isnull().sum().sum()
print(f"\n🎯 Final missing values: {final_missing}")

if final_missing == 0:
    print("✅ Data quality validation PASSED")
else:
    print("⚠️ Data quality validation: Some issues remain")


🔹 STEP 8: Validate Data Quality
----------------------------------------
🔍 Running final data quality checks...

📋 Final Data Types:
Segment                        object
Country                        object
Product                        object
Discount_Band                  object
Units_Sold                    float64
Manufacturing_Price           float64
Sale_Price                    float64
Gross_Sales                   float64
Discounts                     float64
Sales                         float64
COGS                          float64
Profit                        float64
Date                   datetime64[ns]
Month_Number                    int64
Month_Name                     object
Year                            int32
Profit_Margin                 float64
Cost_Ratio                    float64
Revenue_per_Unit              float64
Discount_Percentage           float64
dtype: object

🔍 Duplicate rows: 0

✅ Logical Consistency Checks:
   Profit consistency: Average differenc

In [14]:
print("\n🔹 STEP 9: Save Cleaned Dataset")
print("-" * 40)

# 9.1 Save to CSV
output_path = "../data/financial_data_cleaned.csv"

try:
    df_clean.to_csv(output_path, index=False)
    print(f"✅ Cleaned dataset saved to: {output_path}")
    
    # Verify file was created
    file_size = os.path.getsize(output_path) / 1024 / 1024  # Size in MB
    print(f"📁 File size: {file_size:.2f} MB")
    
    # Verify we can load it back
    verify_df = pd.read_csv(output_path)
    print(f"✅ Verification: Successfully loaded {len(verify_df)} records")
    
except Exception as e:
    print(f"❌ Error saving dataset: {e}")

# 9.2 Final Summary
print("\n" + "=" * 60)
print("🎉 DATA CLEANING PROCESS COMPLETED SUCCESSFULLY!")
print("=" * 60)

print(f"📊 Final Dataset Info:")
print(f"   • Records: {len(df_clean)}")
print(f"   • Columns: {len(df_clean.columns)}")

# Safe date range display
try:
    if 'Date' in df_clean.columns and pd.api.types.is_datetime64_any_dtype(df_clean['Date']):
        date_min = df_clean['Date'].min()
        date_max = df_clean['Date'].max()
        print(f"   • Date Range: {date_min.strftime('%Y-%m-%d')} to {date_max.strftime('%Y-%m-%d')}")
    else:
        print(f"   • Date Range: Date column not available or not datetime")
except Exception as e:
    print(f"   • Date Range: Error displaying date range")

# Safe numeric calculations
try:
    if 'Sales' in df_clean.columns and pd.api.types.is_numeric_dtype(df_clean['Sales']):
        total_sales = df_clean['Sales'].sum()
        print(f"   • Total Sales: ${total_sales:,.2f}")
    else:
        print(f"   • Total Sales: Sales column not available or not numeric")
except Exception as e:
    print(f"   • Total Sales: Error calculating sales")

try:
    if 'Profit' in df_clean.columns and pd.api.types.is_numeric_dtype(df_clean['Profit']):
        total_profit = df_clean['Profit'].sum()
        print(f"   • Total Profit: ${total_profit:,.2f}")
    else:
        print(f"   • Total Profit: Profit column not available or not numeric")
except Exception as e:
    print(f"   • Total Profit: Error calculating profit")

print(f"\n📈 Key KPIs Created:")
new_features = [col for col in df_clean.columns if col not in df.columns]

for feature in new_features:
    if feature in df_clean.columns:
        try:
            # Only calculate mean for numeric columns
            if pd.api.types.is_numeric_dtype(df_clean[feature]):
                avg_value = df_clean[feature].mean()
                print(f"   • {feature}: {avg_value:.2f}")
            else:
                # For categorical columns, show value counts
                unique_count = df_clean[feature].nunique()
                print(f"   • {feature}: {unique_count} unique categories")
        except Exception as e:
            print(f"   • {feature}: Error calculating statistics")

# Additional: Show data types summary
print(f"\n📋 Data Types Summary:")
print(f"   • Numeric columns: {len(df_clean.select_dtypes(include=['int64', 'float64']).columns)}")
print(f"   • Categorical columns: {len(df_clean.select_dtypes(include=['object', 'category']).columns)}")
print(f"   • Date columns: {len(df_clean.select_dtypes(include=['datetime64']).columns)}")

print(f"\n🎯 Next Step: Run '02_analysis_visuals.ipynb' for EDA!")


🔹 STEP 9: Save Cleaned Dataset
----------------------------------------
✅ Cleaned dataset saved to: ../data/financial_data_cleaned.csv
📁 File size: 0.12 MB
✅ Verification: Successfully loaded 700 records

🎉 DATA CLEANING PROCESS COMPLETED SUCCESSFULLY!
📊 Final Dataset Info:
   • Records: 700
   • Columns: 20
   • Date Range: 2013-01-09 to 2014-01-12
   • Total Sales: $118,726,350.29
   • Total Profit: $17,671,023.54

📈 Key KPIs Created:
   • Discount_Band: 4 unique categories
   • Units_Sold: 1608.29
   • Manufacturing_Price: 96.48
   • Sale_Price: 118.43
   • Gross_Sales: 182759.43
   • Month_Number: 7.90
   • Month_Name: 12 unique categories
   • Profit_Margin: 28.46
   • Cost_Ratio: 72.10
   • Revenue_per_Unit: 109.80
   • Discount_Percentage: 7.33

📋 Data Types Summary:
   • Numeric columns: 13
   • Categorical columns: 5
   • Date columns: 1

🎯 Next Step: Run '02_analysis_visuals.ipynb' for EDA!
